In [12]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.3.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [13]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [15]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Read from S3") \
    .getOrCreate()

# Download the CSV file to local storage
!wget -O home_sales_revised.csv "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Read data from the downloaded file into DataFrame
df = spark.read.csv("home_sales_revised.csv", header=True, inferSchema=True)

# Show the DataFrame schema and some sample data
df.printSchema()
df.show(5)



--2024-02-13 01:03:49--  https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv
Resolving 2u-data-curriculum-team.s3.amazonaws.com (2u-data-curriculum-team.s3.amazonaws.com)... 52.217.174.41, 16.182.70.57, 52.217.114.41, ...
Connecting to 2u-data-curriculum-team.s3.amazonaws.com (2u-data-curriculum-team.s3.amazonaws.com)|52.217.174.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2747874 (2.6M) [text/csv]
Saving to: ‘home_sales_revised.csv’

home_sales_revised. 100%[===================>]   2.62M  6.13MB/s    in 0.4s    

2024-02-13 01:03:50 (6.13 MB/s) - ‘home_sales_revised.csv’ saved [2747874/2747874]

root
 |-- id: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer 

In [16]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("home_sales_view")



In [18]:
# SQL query to calculate average price for a four-bedroom house sold in each year
avg_price_query = """
    SELECT
        YEAR(date) AS year,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales_view
    WHERE
        bedrooms = 4
    GROUP BY
        YEAR(date)
    ORDER BY
        year
"""

# Execute the SQL query
avg_price_result = spark.sql(avg_price_query)

# Show the result
avg_price_result.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [20]:
# SQL query to calculate average price for homes built in each year, having 3 bedrooms and 3 bathrooms
avg_price_query = """
    SELECT
        date_built AS year_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales_view
    WHERE
        bedrooms = 3 AND bathrooms = 3
    GROUP BY
        date_built
    ORDER BY
        date_built
"""

# Execute the SQL query
avg_price_result = spark.sql(avg_price_query)

# Show the result
avg_price_result.show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [21]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?


avg_price_query = """
    SELECT
        date_built AS year_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales_view
    WHERE
        bedrooms = 3 AND
        bathrooms = 3 AND
        floors = 2 AND
        sqft_living >= 2000
    GROUP BY
        date_built
    ORDER BY
        date_built
"""

# Execute the SQL query
avg_price_result = spark.sql(avg_price_query)

# Show the result
avg_price_result.show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [22]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

import time

# Start time
start_time = time.time()

# SQL query to calculate the average view rating for homes with price >= $350,000
avg_view_query = """
    SELECT
        ROUND(AVG(view), 2) AS avg_view_rating
    FROM
        home_sales_view
    WHERE
        price >= 350000
"""

# Execute the SQL query
avg_view_result = spark.sql(avg_view_query)

# Show the result
avg_view_result.show()

# Runtime
print("--- %s seconds ---" % (time.time() - start_time))


+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- 0.6496336460113525 seconds ---


In [24]:
# 7. Cache the the temporary table home_sales.

df.createOrReplaceTempView("home_sales_view")

# Cache the temporary view home_sales_view
spark.catalog.cacheTable("home_sales_view")


In [25]:
# Check if the temporary view home_sales_view is cached
is_cached = spark.catalog.isCached('home_sales_view')

# Print the result
print("Is home_sales_view cached:", is_cached)


Is home_sales_view cached: True


In [26]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.


start_time_cached = time.time()

# Run the query using the cached data
cached_query_result = spark.sql("""
    SELECT *
    FROM home_sales_view
    WHERE price >= 350000
""")

# Show the result
cached_query_result.show()

# Runtime for cached query
cached_runtime = time.time() - start_time_cached
print("--- Cached Runtime: %s seconds ---" % cached_runtime)


# Start time for uncached query
start_time_uncached = time.time()

# Run the same query without using cached data
uncached_query_result = spark.sql("""
    SELECT *
    FROM home_sales_view
    WHERE price >= 350000
""")

# Show the result
uncached_query_result.show()

# Runtime for uncached query
uncached_runtime = time.time() - start_time_uncached
print("--- Uncached Runtime: %s seconds ---" % uncached_runtime)



+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|131492a1-72e2-4a8...|2020-02-08 00:00:00|      2017|419199|       2|        3|       2062|    8876|     2|         0|   6|
|f1e4cef

In [27]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

output_path = "partitioned_home_sales_parquet"

# Write the DataFrame to Parquet format partitioned by the "date_built" field
df.write.partitionBy("date_built").parquet(output_path)

# Show the saved files
!ls -l {output_path}


total 32
drwxr-xr-x 2 root root 4096 Feb 13 01:18 'date_built=2010'
drwxr-xr-x 2 root root 4096 Feb 13 01:18 'date_built=2011'
drwxr-xr-x 2 root root 4096 Feb 13 01:18 'date_built=2012'
drwxr-xr-x 2 root root 4096 Feb 13 01:18 'date_built=2013'
drwxr-xr-x 2 root root 4096 Feb 13 01:18 'date_built=2014'
drwxr-xr-x 2 root root 4096 Feb 13 01:18 'date_built=2015'
drwxr-xr-x 2 root root 4096 Feb 13 01:18 'date_built=2016'
drwxr-xr-x 2 root root 4096 Feb 13 01:18 'date_built=2017'
-rw-r--r-- 1 root root    0 Feb 13 01:18  _SUCCESS


In [28]:
# 11. Read the parquet formatted data.

parquet_path = "partitioned_home_sales_parquet"

# Read the Parquet formatted data
parquet_df = spark.read.parquet(parquet_path)

# Show the schema of the DataFrame
parquet_df.printSchema()

# Show the first few rows of the DataFrame
parquet_df.show(5)



root
 |-- id: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- date_built: integer (nullable = true)

+--------------------+-------------------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|               date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+-------------------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06 00:00:00|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09 00:00:00|229896|       3

In [29]:
# 12. Create a temporary table for the parquet data.

parquet_df.createOrReplaceTempView("parquet_home_sales_view")


In [30]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.



# Start time for Parquet query
start_time_parquet = time.time()

# Run the query using the Parquet DataFrame
parquet_query_result = spark.sql("""
    SELECT ROUND(AVG(view), 2) AS avg_view_rating
    FROM parquet_home_sales_view
    WHERE price >= 350000
""")

# Show the result
parquet_query_result.show()

# Runtime for Parquet query
parquet_runtime = time.time() - start_time_parquet
print("--- Parquet Runtime: %s seconds ---" % parquet_runtime)


+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- Parquet Runtime: 0.5833420753479004 seconds ---


In [32]:
# 14. Uncache the home_sales temporary table.

spark.catalog.uncacheTable("home_sales_view")


In [33]:
# 15. Check if the home_sales is no longer cached


is_cached = spark.catalog.isCached('home_sales_view')

# Print the result
print("Is home_sales_view cached:", is_cached)


Is home_sales_view cached: False
